In [1]:
import sys, os
from scipy import sparse
from scipy.sparse.linalg import spsolve
from scipy.sparse import eye
import numpy as np

from core.InputOutput import *
from core.MeshDisplay import MeshDisplay
from core.HalfEdgeMesh import *

In [2]:
filename = 'meshes/bunny.obj'
mesh = HalfEdgeMesh(readMesh(filename),staticGeometry=False)

Reading mesh from: meshes/cube.obj
   Loaded mesh with 6,146 vertices and 12,288 faces

Constructing HalfEdge Mesh...
  Input mesh has 0 unpaired edges (which only appear in one direction)
  Input mesh has 0 unpaired verts (which touch some unpaired edge)
  Filled 0 boundary holes in mesh using imaginary halfedges/faces
HalfEdge mesh construction completed
=== HalfEdge mesh statistics:
    Halfedges = 36864  (+ 0 imaginary)
    Edges = 18432
    Faces = 12288  (+ 0 imaginary)
    Verts = 6146
    - Max vertex degree = 8
    - Min vertex degree = 3
    - n boundary verts = 0


In [4]:
def smoothing(mesh, iterations=1):
    
    verts = list(mesh.verts)
    vert_ids = {i: n for n, i in enumerate(verts)}

    for iteration in range(iterations):

        data = []
        col_ind = []
        row_ind = []

        for n, i in enumerate(verts):
            cotan_sum = sum(j.cotan + j.twin.cotan for j in i.adjacentHalfEdges())
            data.append(cotan_sum / 2.)
            row_ind.append(n)
            col_ind.append(n)

        for m in mesh.halfEdges:
            data.append(-(m.cotan + m.twin.cotan) / 2.)
            row_ind.append(vert_ids[m.vertex])
            col_ind.append(vert_ids[m.next.vertex])

        # construct the laplacian matrix
        L = sparse.coo_matrix((data, (row_ind, col_ind)))
        I = eye(len(verts))

        # calculate new positions
        new_positions = spsolve(I + L, [i.position for i in verts])
        
        # update mesh vertices positions
        for v, new_position in zip(verts, new_positions):
            v.position = new_position
            
        print('Done %d iteration' % iteration)

    return mesh

In [ ]:
smoothing(mesh, 10)
meshDisplay = MeshDisplay()
meshDisplay.setMesh(mesh)
meshDisplay.startMainLoop()

Done 0 iteration
Done 1 iteration
Done 2 iteration
Done 3 iteration
Done 4 iteration
Done 5 iteration
Done 6 iteration
Done 7 iteration
Done 8 iteration
Done 9 iteration
Creating MeshDisplay window
  Platform is: darwin
  init'd GLUT
  init'd GL
Starting MeshDisplay openGL main loop
Filling buffers for visualization
Done filling buffers for visualization
